In [1]:


import numpy as np
import pandas as pd

from util.reselience_metrics import (
    EvalWindows,
    to_performance,
    compute_all,
)


In [ ]:
# === PATH TO YOUR RUN FILES (.npz) ===
PATH_PERT = ""     # run WITH perturbation
PATH_BASE = ""              # control run WITHOUT perturbation

# time axis (must exist in BOTH files)
TIME_KEY = "t"

# Metric keys (one pair per performance metric you want to evaluate)
KEYMAP = {
    "speed":  ("avg_speed", "avg_speed"),        # (pert_key, base_key)
    "queue":  ("total_queue", "total_queue"),
    "wait":   ("total_wait", "total_wait"),
    "ret": ("avg_reward", "avg_reward"),
}

# Which metrics to evaluate in this run
SELECTED = ["speed", "queue", "wait"]  # or ["speed"] or add "reward" if meaningful

# === Windows (seconds or steps; must match your time units) ===
wins = EvalWindows(
    pre_start=0,  # start of pre-shock window
    t0=600,         # shock start
    t1=1800,         # shock end
    post_end=3600    # end of evaluation horizon
)

# Normalization rule baked in our util:
# - higher_is_better=True  -> minmax
# - higher_is_better=False -> inv1p
DIRECTION = {
    "speed":  True,   # higher is better
    "reward": True,   # often higher is better (if your reward is aligned that way)
    "queue":  False,  # lower is better
    "wait":   False,  # lower is better
}


In [ ]:
def array_to_series(t: np.ndarray, x: np.ndarray) -> pd.Series:
    """Create a pandas Series indexed by time/step."""
    if x.shape != t.shape:
        raise ValueError(f"Shape mismatch: t{t.shape} vs x{x.shape}")
    return pd.Series(x.astype(float), index=t.astype(float)).sort_index()

In [ ]:
# Load files
pert = np.load(PATH_PERT)
base = np.load(PATH_BASE)

# Extract time axes
t_pert = pert[TIME_KEY]
t_base = base[TIME_KEY]

# For convenience, build a dict of series per metric
series_pert = {}
series_base = {}

for name in SELECTED:
    k_pert, k_base = KEYMAP[name]
    x_pert = pert[k_pert]
    x_base = base[k_base]
    series_pert[name] = array_to_series(t_pert, x_pert)
    series_base[name] = array_to_series(t_base, x_base)


In [ ]:
rows = []
for name in SELECTED:
    higher_is_better = DIRECTION[name]

    # 1) Normalize perturbed run into performance P(t)
    P = to_performance(
        series_pert[name],
        higher_is_better=higher_is_better,
        ref_window=wins.pre,   # normalize on pre window for comparability
        allow_clip=False       # keep excursions beyond [0,1]
    )

    # 2) Compute resilience metrics using baseline raw series
    res = compute_all(
        P, wins,
        baseline_raw=series_base[name],
        higher_is_better=higher_is_better,
        target=0.90,           # RT90
        band=0.05              # ±5% settling band
    )
    res["metric"] = name
    rows.append(res)

res_df = pd.DataFrame(rows).set_index("metric")
res_df
